# House MD Names analysis

I've recently fallen back into an House M.D. 10 years after my last rewatch (when I was 14). Partway through my rewatch I wanted to know if Cuddy ever addressed Wilson by "James", which escalated to wanting to know how many times every character mentioned every other character by their first or last name.

In [211]:
import re
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
from urllib.parse import urlparse

## Data extraction

In [26]:
# Transcript list
# https://clinic-duty.livejournal.com/12225.html

response = requests.get('https://clinic-duty.livejournal.com/12225.html')
assert response.ok

In [209]:
episode_elements[10]['href']

'http://community.livejournal.com/clinic_duty/1970.html'

In [227]:
soup = BeautifulSoup(response.text, 'html.parser')
episode_elements = soup.select('div.entryHolder a[target="_blank"][href]')
episode_titles = re.findall(r'(\d\.\d\d) - (.+?)(?=(\d\.\d\d)|$)', episode_elements[0].parent.text.replace('\n', ''))
episode_titles = [(x[0], x[1]) for x in episode_titles]

episode_df = pd.DataFrame(episode_titles, columns=['#', 'Episode Title'])
episode_df['Season'] = episode_df['#'].apply(lambda x: x[0])
episode_df['Episode Number'] = episode_df['#'].apply(lambda x: int(x[-2:]))
episode_df.set_index('#', inplace=True)


episode_df.loc['3.24', 'Episode Title'] = " ".join(episode_df.loc['3.24', 'Episode Title'].split()[:2])
episode_df.loc['6.22', 'Episode Title'] = " ".join(episode_df.loc['6.22', 'Episode Title'].split()[:2])
episode_df['Episode Title'] = episode_df['Episode Title'].str.strip()

episode_urls = {elem.text.strip(): elem['href'] for elem in episode_elements}
episode_df['URL'] = episode_df['Episode Title'].apply(lambda x: episode_urls[x])

def url_parsing(x):
    url = urlparse(x)
    path = url.path
    path = re.sub('clinic_duty/', '', path)
    return url._replace(scheme='https', netloc='clinic_duty.livejournal.com', path=path).geturl()

episode_df['URL'] = episode_df['URL'].apply(lambda x: url_parsing(x))

display(episode_df)

,Episode Title,Season,Episode Number,URL
#,,,,
1.01,Pilot,1,1,https://clinic_duty.livejournal.com/385.html
1.02,Paternity,1,2,https://clinic_duty.livejournal.com/620.html
1.03,Occam's Razor,1,3,https://clinic_duty.livejournal.com/1018.html
1.04,Maternity,1,4,https://clinic_duty.livejournal.com/1082.html
1.05,Damned If You Do,1,5,https://clinic_duty.livejournal.com/1302.html
...,...,...,...,...
8.18,Body and Soul,8,18,https://clinic_duty.livejournal.com/49746.html...
8.19,The C-Word,8,19,https://clinic_duty.livejournal.com/49991.html...
8.20,Post Mortem,8,20,https://clinic_duty.livejournal.com/50246.html...


In [228]:
# episode_df.to_csv('episode_data.csv', index=True)

## Data analysis

## Conclusions